In [1]:
import sys
import smaract.ctl as ctl




In [2]:
buffer = ctl.FindDevices()
print(buffer)

usb:sn:MCS2-00002553


In [3]:
d_handle = None
# try:
d_handle = ctl.Open(buffer)
print("MCS2 opened {}.".format(buffer))

MCS2 opened usb:sn:MCS2-00002553.


In [4]:
channel = 2

In [5]:
def calibrate(channel):
    print("MCS2 start calibration on channel: {}.".format(channel))
    # Set calibration options (start direction: forward)
    ctl.SetProperty_i32(d_handle, channel, ctl.Property.CALIBRATION_OPTIONS, 0)
    # Start calibration sequence
    ctl.Calibrate(d_handle, channel)

In [6]:
calibrate(channel)

MCS2 start calibration on channel: 2.


In [7]:
def move(channel, move_mode, direction):
    # Set move mode depending properties for the next movement.
    if move_mode == ctl.MoveMode.CL_ABSOLUTE:
        # Set move velocity [in pm/s].
        ctl.SetProperty_i64(d_handle, channel, ctl.Property.MOVE_VELOCITY, 1000000000)
        # Set move acceleration [in pm/s2].
        ctl.SetProperty_i64(d_handle, channel, ctl.Property.MOVE_ACCELERATION, 1000000000)
        # Specify absolute position [in pm].
        # (For Piezo Scanner channels adjust to valid value within move range, e.g. +-10000000.)
        move_value = 1000000000
        if direction:
            move_value = -2000000000
        print("MCS2 move channel {} to absolute position: {} pm.".format(channel, move_value))
    elif move_mode == ctl.MoveMode.CL_RELATIVE:
        # Set move velocity [in pm/s].
        ctl.SetProperty_i64(d_handle, channel, ctl.Property.MOVE_VELOCITY, 500000000)
        # Set move acceleration [in pm/s2].
        ctl.SetProperty_i64(d_handle, channel, ctl.Property.MOVE_ACCELERATION, 10000000000)
        # Specify relative position distance [in pm] and direction.
        # (For Piezo Scanner channels adjust to valid value within move range, e.g. 10000000.)
        move_value = 500000000
        if direction:
            move_value = -move_value
        print("MCS2 move channel {} relative: {} pm.".format(channel, move_value))
    elif move_mode == ctl.MoveMode.SCAN_ABSOLUTE:
        # Set scan velocity [in dac increments/s].
        # Valid range (Stick-Slip Piezo): 1 to 65535000000
        # Valid range (Piezo Scanner): 1 to 4194300000
        ctl.SetProperty_i64(d_handle, channel, ctl.Property.SCAN_VELOCITY, (65535*2))
        # Specify absolute scan target to which to scan to [in dac increments].
        # Valid range (Stick-Slip Piezo): 0 to 65535 corresponding to 0 to 100V piezo voltage
        # Valid range (Piezo Scanner): 0 to 1048575 corresponding to -20 to 100V piezo voltage
        move_value = 65535
        if direction:
            move_value = 0
        print("MCS2 scan channel {} absolute to: {}.".format(channel, move_value))
    elif move_mode == ctl.MoveMode.SCAN_RELATIVE:
        # Set scan velocity [in dac increments/s].
        ctl.SetProperty_i64(d_handle, channel, ctl.Property.SCAN_VELOCITY, 65535)
        # Specify relative scan target and direction to which to scan to [in dac increments].
        # Valid range( Stick-Slip Piezo): -65535 to 65535 corresponding to 0 to 100V piezo voltage
        # Valid range (Piezo Scanner): -1048575 to 1048575 corresponding to -20 to 100V piezo voltage
        # If the resulting absolute scan target exceeds the valid range the scan movement will stop at the boundary.
        move_value = 65535
        if direction:
            move_value = -move_value
        print("MCS2 scan channel {} relative: {}.".format(channel, move_value))
    elif move_mode == ctl.MoveMode.STEP: # (only Stick-Slip piezo positioners)
        # Set step frequency [in Hz].
        # Valid range: 1 to 20000 Hz
        ctl.SetProperty_i32(d_handle, channel, ctl.Property.STEP_FREQUENCY, 1000)
        # Set maximum step amplitude [in dac increments].
        # valid range: 0 to 65535 corresponding to 0 to 100V piezo voltage
        # Lower amplitude values result in smaller step width.
        ctl.SetProperty_i32(d_handle, channel, ctl.Property.STEP_AMPLITUDE, 65535)
        # Specify the number of steps to perform and the direction.
        move_value = 500
        if direction:
            move_value = -move_value
        print("MCS2 open loop step move, channel {}, steps: {}.".format(channel, move_value))
    # Start actual movement.
    ctl.Move(d_handle, channel, move_value, 0)
    # Note that the function call returns immediately, without waiting for the movement to complete.
    # The "ChannelState.ACTIVELY_MOVING" (and "ChannelState.CLOSED_LOOP_ACTIVE") flag in the channel state
    # can be monitored to determine the end of the movement.


In [27]:
move(channel, ctl.MoveMode., True)



MCS2 open loop step move, channel 1, steps: -500.


In [44]:
move(channel, ctl.MoveMode.CL_ABSOLUTE, False)

MCS2 move channel 5 to absolute position: 1000000000 pm.


In [43]:
channel=5

In [35]:
def findReference(channel):
    print("MCS2 find reference on channel: {}.".format(channel))
    # Set find reference options.
    # The reference options specify the behavior of the find reference sequence.
    # The reference flags can be ORed to build the reference options.
    # By default (options = 0) the positioner returns to the position of the reference mark.
    # Note: In contrast to previous controller systems this is not mandatory.
    # The MCS2 controller is able to find the reference position "on-the-fly".
    # See the MCS2 Programmer Guide for a description of the different modes.
    ctl.SetProperty_i32(d_handle, channel, ctl.Property.REFERENCING_OPTIONS, 0)
    # Set velocity to 1mm/s
    ctl.SetProperty_i64(d_handle, channel, ctl.Property.MOVE_VELOCITY, 1000000000)
    # Set acceleration to 10mm/s2.
    ctl.SetProperty_i64(d_handle, channel, ctl.Property.MOVE_ACCELERATION, 10000000000)
    # Start referencing sequence
    ctl.Reference(d_handle, channel)
    # Note that the function call returns immediately, without waiting for the movement to complete.
    # The "ChannelState.REFERENCING" flag in the channel state can be monitored to determine
    # the end of the referencing sequence.

In [22]:
findReference(channel)

MCS2 find reference on channel: 1.


In [23]:
findReference(0)

MCS2 find reference on channel: 0.


In [24]:
findReference(2)

MCS2 find reference on channel: 2.


In [25]:
findReference(5)

MCS2 find reference on channel: 5.


In [19]:
if d_handle != None:
        ctl.Close(d_handle)